- load namelist and prepare
- train model(s) male/female
- filter results for "unkown" names
- store results
- (batch) send to msg-api

In [56]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from IPython.display import clear_output
import collections
import urllib
from os import listdir
from os.path import isfile, join
import pandas
import re

In [138]:
#source: https://daten.berlin.de/datensaetze/liste-der-h%C3%A4ufigen-vornamen-2015

fileurllist = ['http://www.berlin.de/daten/liste-der-vornamen-2015/charlottenburg-wilmersdorf.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/friedrichshain-kreuzberg.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/lichtenberg.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/marzahn-hellersdorf.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/mitte.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/neukoelln.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/pankow.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/reinickendorf.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/spandau.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/steglitz-zehlendorf.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/tempelhof-schoeneberg.csv',
              'http://www.berlin.de/daten/liste-der-vornamen-2015/treptow-koepenick.csv']
!mkdir name-csvs/
for url in fileurllist:
    file_name = url.split('/')[-1]
    full_path = 'name-csvs/' + file_name
    urllib.urlretrieve(url, full_path)

Build a proper text file for training

In [217]:
def build_training_data(gender="w", districts="all", weighted=False):
    # returns one string, names separated by \n, names randomized, repetitions across districts NOT removed
    raw_names = ''
    if (districts == "all"): 
        fileslist = [f for f in listdir('name-csvs/') if isfile(join('name-csvs/', f))]
        print "loading all districts in folder :"  + str(files_in_dir)
    
    else:#better check if is list
        print "loading districts as specified in list: " + str(districts)
        fileslist = districts
        
    for idx,current_file in enumerate(fileslist):
        temp_df = pandas.read_csv(join('name-csvs/', current_file),';')     
        if idx == 0:
            full_df = temp_df
            
        if (weighted == True):
            df_append = temp_df
            for iteration in range(max(temp_df.anzahl)):
                df_append = df_append.append(temp_df.loc[temp_df.anzahl>=iteration])
            temp_df = df_append
            
        full_df = full_df.append(temp_df)
    
    full_df.reset_index(drop = True, inplace = True)
    raw_names = re.sub(pattern = r' |\d',repl='', string = full_df.reindex(np.random.permutation(full_df.index)).loc[full_df['geschlecht'] == gender].vorname.to_string())
 
    return raw_names, full_df

In [145]:
my_districts = ['lichtenberg.csv', 'marzahn-hellersdorf.csv', 'neukoelln.csv', 'reinickendorf.csv']

In [144]:
names_w_all_districts, _  = build_training_data()

loading all districts (in folder)


In [218]:
names_w_my_districts, test_df = build_training_data(gender = 'w', districts= my_districts, weighted=True)

loading districts as specified in list: ['lichtenberg.csv', 'marzahn-hellersdorf.csv', 'neukoelln.csv', 'reinickendorf.csv']


In [219]:
len(names_w_my_districts)

107037

In [220]:
test_df.shape

(31432, 3)